## Using Santander Cycling data (Random forest) to predict bike occupancy for December.
https://cycling.data.tfl.gov.uk/


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPolygon

%matplotlib inline

import json
from xml.etree import ElementTree as ET
import requests
from io import BytesIO
from io import StringIO

from zipfile import ZipFile
import urllib.request
from urllib.request import urlopen
import os
from pandas.io.json import json_normalize
from itertools import chain, starmap


In [2]:
bike_data_2012_zip = urllib.request.urlopen("https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2012.zip")
bike_data_2013_zip = urllib.request.urlopen("https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2013.zip")
bike_data_2014_zip = urllib.request.urlopen("https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip")
bike_data_2015_zip = urllib.request.urlopen("https://cycling.data.tfl.gov.uk/usage-stats/2015TripDatazip.zip")
bike_data_2016_zip = urllib.request.urlopen("https://cycling.data.tfl.gov.uk/usage-stats/2016TripDataZip.zip")

In [307]:
import shutil
import zipfile
def encoding_zipfile(url):    
    with ZipFile(BytesIO(url.read())) as zipfile:
        for fileinfo in zipfile.infolist():
            filename = fileinfo.filename.encode('cp437').decode('gbk')
            print(filename)

In [299]:
bike_data_2013 = encoding_zipfile(bike_data_2013_zip)

1. Journey Data Extract 01Jan-05Jan13.csv
10. Journey Data Extract 18Aug-13Sep13.csv
11. Journey Data Extract 14Sep13-12Oct13.csv
12. Journey Data Extract 13Oct13-09Nov13.csv
13. Journey Data Extract 10Nov13-07Dec13.csv
14. Journey Data Extract 08Dec13-04Jan14.csv
2. Journey Data Extract 06Jan-02Feb13.csv
3. Journey Data Extract 03Feb-02Mar13.csv
4. Journey Data Extract 03Mar-31Mar13.csv
5. Journey Data Extract 01Apr-27Apr13.csv
6. Journey Data Extract 28Apr-25May13.csv
7. Journey Data Extract 26May-22Jun13.csv
8. Journey Data Extract 23Jun-20Jul13.csv
9. Journey Data Extract 21Jul-17Aug13.csv


In [ ]:
import csv

def unicode_csv_reader(utf8_data, dialect=csv.excel, **kwargs):
    csv_reader = csv.reader(utf8_data, dialect=dialect, **kwargs)
    for row in csv_reader:
        yield [unicode(cell, 'utf-8') for cell in row]

In [3]:
# Define function to parse zipfiles to dataframe
def zipfile_to_df(url):    
    with ZipFile(BytesIO(url.read())) as zipfile:
        dfs = []
        for col_name in zipfile.namelist():
            with zipfile.open(col_name) as csv:
                df = pd.read_csv(csv, index_col=None, header=0)
                dfs.append(df)
    data = pd.concat(dfs, ignore_index=True)
    return data

In [4]:
#Applying read function to 2015 and 2016 bike data
bike_2012_df = zipfile_to_df(bike_data_2012_zip)
#bike_2013_df = zipfile_to_df(bike_data_2013_zip)
#bike_2014_df = zipfile_to_df(bike_data_2014_zip)
bike_2015_df = zipfile_to_df(bike_data_2015_zip)
bike_2016_df = zipfile_to_df(bike_data_2016_zip)

/Users/admin/bike_data_analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/admin/bike_data_analysis/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [5]:
#Cleaning incorrect results on 2012 dataframe
bike_2012_df = bike_2012_df[bike_2012_df['StartStation Id'] != 'Tabletop1']

In [6]:
#Concatenating all dataframes that can be read in cleanly
bike_data = pd.concat([bike_2012_df,bike_2015_df,bike_2016_df], ignore_index=True).drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 9','EndStation Logical Terminal','StartStation Logical Terminal','endStationPriority_id'], axis=1)

/Users/admin/bike_data_analysis/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
def clean_bike_data(dataframe):
    print("Original shape: ",dataframe.shape)
    dataframe.dropna(axis=0, subset=["StartStation Id", "EndStation Id", "Start Date", "End Date"], inplace=True)
    print("Shape after dropping Na results: ",dataframe.shape)
    
    dataframe["EndStation Id"] = dataframe["EndStation Id"].astype(int)
    dataframe["StartStation Id"] = dataframe["StartStation Id"].astype(int)
    
    dataframe = dataframe[dataframe["StartStation Id"] != dataframe["EndStation Id"]]
    dataframe = dataframe.loc[:,('Start Date',
                           'StartStation Id',
                           'End Date',
                           'EndStation Id',
                           'Duration')]                 
    print("Shape after removing trips which start and end at the same location", dataframe.shape)
    ## Extra drop for duplicates
    dataframe.drop_duplicates(inplace=True)
    print("Shape after removing duplicates: ",dataframe.shape)

In [9]:
clean_bike_data(bike_data)

Original shape:  (29267602, 9)
Shape after dropping Na results:  (28933309, 9)
Shape after removing trips which start and end at the same location (27917787, 5)
Shape after removing duplicates:  (26858182, 5)


In [8]:
bike_locations = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

In [9]:
#url containing the locations of all Santander bike stops
bike_locations = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

def get_location_from_xml(url):
    response = requests.get(url)
    root = ET.fromstring(response.content)
    
    col_list = []
    for i in range(0, len(root)):
        id_list = int(root[i][0].text)
        name_list = str(root[i][1].text)
        lat_list = float(root[i][3].text)
        lon_list = float(root[i][4].text)
        capacity_list = int(root[i][12].text)
        col_list.append([id_list,name_list,lat_list,lon_list,capacity_list])
    
    all_locs = pd.DataFrame(list(col_list), columns = ["name","id","lat","lon","capacity"])
#    all_locs.to_dataframe(url.split('/')[7].split('.')[0]+'.csv', header=True, index=None)
    print("Shape of cycle hire locations: ",all_locs.shape)
    
    return all_locs

bike_locations = get_location_from_xml(bike_locations)

In [12]:
bike_data_locations = bike_data.merge(bike_locations, left_on=['StartStation Id'], right_on=['name'], suffixes=('_s')).drop(columns='id')
bike_data_locations.rename(columns={"name": "sname", 
                                    "lat": "slat",
                                    "lon": "slon",
                                    "capacity": "scap"
                                   })

,Bike Id,Duration,End Date,EndStation Id,EndStation Name,Rental Id,Start Date,StartStation Id,StartStation Name,name,lat,lon,capacity
0,893.0,1238.0,04/01/2012 00:20,169.0,Porchester Place: Paddington,9340768,04/01/2012 00:00,224,Whiteley's: Bayswater,224,51.51031,-0.187402,17
1,6603.0,19.0,04/01/2012 07:37,224.0,Whiteley's: Bayswater,9363259,04/01/2012 07:37,224,Whiteley's: Bayswater,224,51.51031,-0.187402,17
2,3992.0,111.0,04/01/2012 07:39,176.0,Gloucester Terrace: Bayswater,9355931,04/01/2012 07:37,224,Whiteley's: Bayswater,224,51.51031,-0.187402,17
3,2699.0,373.0,04/01/2012 08:16,224.0,Whiteley's: Bayswater,9357327,04/01/2012 08:10,224,Whiteley's: Bayswater,224,51.51031,-0.187402,17
4,6603.0,707.0,04/01/2012 08:26,428.0,Exhibition Road: Knightsbridge,9360881,04/01/2012 08:14,224,Whiteley's: Bayswater,224,51.51031,-0.187402,17


In [ ]:
bike_data_locations.merge(bike_data_locations, left_on=['EndStation Id'], right_on=['name'], suffixes=('_e')).drop(columns='id')


In [17]:
bike_data_locations.head()

,Bike Id,Duration,End Date,EndStation Id,EndStation Name,Rental Id,Start Date,StartStation Id,StartStation Name,name,lat,lon,capacity
0,893.0,1238.0,04/01/2012 00:20,169,Porchester Place: Paddington,9340768,04/01/2012 00:00,224,Whiteley's: Bayswater,169,51.514746,-0.165164,18
1,4780.0,792.0,04/01/2012 07:35,169,Porchester Place: Paddington,9352469,04/01/2012 07:21,16,Cartwright Gardens : Bloomsbury,169,51.514746,-0.165164,18
2,4999.0,1024.0,04/01/2012 08:23,169,Porchester Place: Paddington,9355146,04/01/2012 08:06,64,William IV Street: Strand,169,51.514746,-0.165164,18
3,3632.0,1120.0,04/01/2012 08:35,169,Porchester Place: Paddington,9358584,04/01/2012 08:16,229,Whitehall Place: Strand,169,51.514746,-0.165164,18
4,2332.0,1416.0,04/01/2012 09:00,169,Porchester Place: Paddington,9356030,04/01/2012 08:37,113,Gloucester Road (Central): South Kensington,169,51.514746,-0.165164,18


In [ ]:
## Generate list of unique routes
unq_locs = all_data.loc[:,('StartStation Id',
                      'EndStation Id')]
print(unq_locs.shape)
unq_locs.drop_duplicates(inplace=True)
print(unq_locs.shape)

## Merge on the lat/lons

unq_locs = unq_locs.merge(right = all_locs,
                             how = 'inner',
                             left_on = 'StartStation Id',
                             right_on = 'id')

print(unq_locs.shape)

unq_locs.drop(labels = ["id", "name"], axis=1, inplace=True)
unq_locs.rename(columns={'lat': 'StartStation lat', 'lon': 'StartStation lon', 
                            'capacity': 'StartStation capacity'},
                   inplace=True)
# Merge end
unq_locs = unq_locs.merge(right = all_locs,
                             how = 'inner',
                             left_on = 'EndStation Id',
                             right_on = 'id')

unq_locs.drop(labels = ["id", "name"], axis=1, inplace=True)
unq_locs.rename(columns={'lat': 'EndStation lat', 'lon': 'EndStation lon',
                           'capacity': 'EndStation capacity'},
                   inplace=True)


print(unq_locs.shape)
unq_locs.head()